<a href="https://colab.research.google.com/github/mralamdari/Machine_Learning_Projects/blob/main/Mnist_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision

In [ ]:
input_size=784
hidden_size=400   #hidden_size= (input_size + out_size) // 2
out_size=10
epochs=10
batch_size=100
learning_rate=0.001

In [ ]:
mean_gray = 0.1307
std_grey = 0.3081

transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize((mean_gray, ), (std_grey, ))])

In [ ]:
train_ds = torchvision.datasets.MNIST(root='/data',
                           train = True,
                           transform = torchvision.transforms.ToTensor(),
                           download=True)

test_ds = torchvision.datasets.MNIST(root='/data',
                                  train=False,
                                  transform = torchvision.transforms.ToTensor(),
                                  download=True)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_ds, 
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
    self.batchnorm1 = nn.BatchNorm2d(8)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=2)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.batchnorm2 = nn.BatchNorm2d(32)
    self.fc1 = nn.Linear(32*7*7, 600)
    self.dropout = nn.Dropout(p=0.5)
    self.fc2 = nn.Linear(600, 100)

  def forward(self, x):
    out = self.conv1(x)
    out = self.batchnorm1(out)
    out = self.relu(out)  
    out = self.maxpool(out)
    out = self.conv2(out)
    out = self.batchnorm2(out)
    out = self.relu(out)
    out = self.maxpool(out)

    out = out.view(-1, 1568)

    out = self.fc1(out)
    out = self.dropout(out)
    out = self.fc2(out)
    return out

In [ ]:
model = Net()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs=20
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

for epoch in range(num_epochs):
  correct = 0
  iterations = 0
  iter_loss = 0.0

  model.train()
  for i, (inputs, labels) in enumerate(train_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = loss_fn(outputs, labels)
    iter_loss += loss.item()
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()

    _, pred = torch.max(outputs, 1)
    correct += (pred == labels).sum().item()
    iterations += 1
  
  train_loss.append(iter_loss/iterations)
  train_accuracy.append(100 * correct/ len(train_ds))
  
  t_loss = 0.0
  correct = 0
  iterations = 0
  model.eval()
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = loss_fn(outputs, labels)
    t_loss += loss.item()
    _, pred = torch.max(outputs, 1) #(100, 10)  ==> 10 is in index 1
    correct += (pred == labels).sum().item()
    iterations += 1

    test_loss.append(t_loss/iterations)
    test_accuracy.append(100*correct/len(test_ds))


  print(f'epoch: {epoch}, Training Loss: {train_loss[-1]:.3f}, Training Accuracy: %{train_accuracy[-1]:.3f}, Test Loss: {test_loss[-1]:.3f}, Test Accuracy:% {test_accuracy[-1]:.3f}')

epoch: 0, Training Loss: 0.119, Training Accuracy: %97.508, Test Loss: 0.131, Test Accuracy:% 97.760
epoch: 1, Training Loss: 0.110, Training Accuracy: %97.672, Test Loss: 0.106, Test Accuracy:% 97.330
epoch: 2, Training Loss: 0.109, Training Accuracy: %97.673, Test Loss: 0.086, Test Accuracy:% 98.390
epoch: 3, Training Loss: 0.115, Training Accuracy: %97.658, Test Loss: 0.145, Test Accuracy:% 97.490
epoch: 4, Training Loss: 0.121, Training Accuracy: %97.523, Test Loss: 0.110, Test Accuracy:% 97.910
epoch: 5, Training Loss: 0.102, Training Accuracy: %97.762, Test Loss: 0.114, Test Accuracy:% 97.890
epoch: 6, Training Loss: 0.112, Training Accuracy: %97.652, Test Loss: 0.096, Test Accuracy:% 97.720
epoch: 7, Training Loss: 0.115, Training Accuracy: %97.557, Test Loss: 0.095, Test Accuracy:% 97.620
epoch: 8, Training Loss: 0.121, Training Accuracy: %97.507, Test Loss: 0.109, Test Accuracy:% 98.030
epoch: 9, Training Loss: 0.117, Training Accuracy: %97.625, Test Loss: 0.144, Test Accuracy

In [ ]:
import sklearn
def standardization(x):
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, random_state=77, test_size=0.2)
x_train_temp = standardization(x_train)
x_test_temp = standardization(x_test)
x_train_std, x_test_std = np.nan_to_num(x_train_temp), np.nan_to_num(x_test_temp)

mnist = sklearn.datasets.fetch_openml('mnist_784', version=1)
x, y = mnist.data, mnist.target
clf = sklearn.ensemble.RandomForestClassifier()
clf.fit(x_train, y_train)
sgd_pred = clf.predict(x_test)
sklearn.metrics.accuracy_score(y_test, sgd_pred)

In [ ]:
import tensorflow as tf

tf.random.set_seed(32)

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
augmented_datgen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255,
                                                                 width_shift_range=0.2, 
                                                                 height_shift_range=0.2,
                                                                 shear_range=0.2,
                                                                 rotation_range=25,
                                                                 zoom_range=0.234)

train_dir = "/content/pizza_steak/train"
test_dir = "/content/pizza_steak/test"


train_data = train_gen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

valid_data = test_gen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

augmented_train = augmented_datgen.flow_from_directory(directory=train_dir, batch_size=32, target_size=(224, 224), class_mode="binary", seed=32)

In [ ]:
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])
# Compile the model
model_1.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

In [ ]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(128, 3, 1, activation="relu")(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(128, 3, 1, activation="relu")(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.GlobalMaxPooling2D()(x)

outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model_1 = tf.keras.Model(inputs, outputs)

model_1.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics="accuracy")

model_1_history=model_1.fit(x_train.reshape((-1 ,28, 28)), y_train, epochs=10, validation_data=(x_test.reshape((-1 ,28, 28)), y_test))
preds = model_1.predict(x_test.reshape(-1, 28, 28, 1))
model_1.evaluate(x_train.reshape(-1, 28, 28, 1), y_train), model_1.evaluate(x_test.reshape(-1, 28, 28, 1), y_test)